In [1]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import os.path
import pandas as pd
import PyPDF2
import requests
from tqdm.contrib.telegram import tqdm

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal
from PyPDF2 import PdfReader

from urllib.parse import unquote
# from transliterate import to_latin
import os

import time
from datetime import datetime
import pytz
import logging



In [2]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [3]:
NEWS_URL = "https://daryo.uz/yangiliklar?page=0&per-page=200"

In [4]:
BASE_URL = "https://daryo.uz" 

In [5]:
# global_links = []
# global_dates = []


def daily_scrapping_urls():
    try:
        links = []
        days = []
        
        resp = requests.get(NEWS_URL)
        if resp.status_code == 200:
            data = resp.text
            soup = bs4.BeautifulSoup(data)
            
            main_content = soup.find('main', class_='maincontent').find_all('div', class_="mini__article-block border")
            
            links.extend([BASE_URL+link.find('a', class_="mini__article-link")['href'] for link in main_content])
            days.extend([link.find('div', class_='article__meta').text.split(",")[0] for link in main_content])

            return {"links":links, "days":days}
        elif resp.status_code != 404:
            time.sleep(60) # 1 minute
    except Exception as e:
        print(e)

In [6]:
def filter_today_news():
    news = daily_scrapping_urls()
    
    df = pd.DataFrame(news)
    df.days = df.days.str.replace("\n\n ", "")
    df.days = df.days.str.strip()
    
    df = df[df['days'] == 'Bugun']
    
    return df.links.to_list()

In [7]:
def daily_scrapping_news():
    try:
        results = []
        
        today_news = filter_today_news()

        for link in today_news:
            resp = requests.get(link)

            if resp.status_code == 200:
                data = resp.text
                soup = bs4.BeautifulSoup(data)
                title = soup.find('div', class_="inner__article-info border").find('h1').text
                content = soup.find('div', class_='default__section border').find_all("p")

                results.append({
                                "title": title.strip(),
                                "content": " ".join([p.text for p in content])
                            })


            elif resp.status_code != 404:
                print("404")
                time.sleep(60) # 1 minute

        return results
    except Exception as e:
        print(e)

In [8]:
def get_values_df():
    values = daily_scrapping_news()
    return pd.DataFrame(values)

In [9]:
def update_df():
    try:
        path = './DailyDaryoUz.pickle'
        check_file = os.path.exists(path)
        if check_file:
            df = pd.read_pickle(path)
        else:
            df = pd.DataFrame()

        current_date = datetime.now(pytz.timezone('Asia/Tashkent'))

        adding_df = get_values_df()
        adding_df.to_pickle(f'daily pickles/{current_date.day}-{current_date.month}-{current_date.year}-daryo.pickle')

        df = pd.concat([df, adding_df], axis=0, ignore_index=True)
        df.to_pickle(path)

        logging.basicConfig(filename='scheduler.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s')
        logging.getLogger().setLevel(logging.INFO)
        logging.info(f"DARYO.uz {current_date} COMPLETE!")

        print(current_date, "COMPLETE!")
    except Exception as e:
        print(e)

In [ ]:
# Schedule Library imported
import schedule
import time


schedule.every().day.at("18:56").do(update_df) # 23:56 UTC+5

# Loop so that the scheduling task
# keeps on running all time.
while True:
 
    # Checks whether a scheduled task
    # is pending to run or not
    schedule.run_pending()
    time.sleep(1)

# 2 months before scrapping


In [41]:
NEWS_URL2 = "https://daryo.uz/yangiliklar?page={page}&per-page=100&last_date=2023-07-27"

In [43]:
try:
    links = []
    days = []
    for i in range(46):
        resp = requests.get(NEWS_URL2.format(page=i))
        if resp.status_code == 200:
            data = resp.text
            soup = bs4.BeautifulSoup(data)

            main_content = soup.find('main', class_='maincontent').find_all('div', class_="mini__article-block border")

            links.extend([BASE_URL+link.find('a', class_="mini__article-link")['href'] for link in main_content])
            days.extend([link.find('div', class_='article__meta').text.split(",")[0] for link in main_content])
        elif resp.status_code != 404:
            time.sleep(60) # 1 minute
except Exception as e:
    print(e)

In [45]:
len(links)

4600

In [47]:
try:
    results = []

#     today_news = filter_today_news()

    for link in links:
        resp = requests.get(link)

        if resp.status_code == 200:
            data = resp.text
            soup = bs4.BeautifulSoup(data)
            title = soup.find('div', class_="inner__article-info border").find('h1').text
            content = soup.find('div', class_='default__section border').find_all("p")

            results.append({
                            "title": title.strip(),
                            "content": " ".join([p.text for p in content])
                        })


        elif resp.status_code != 404:
            print("404")
            time.sleep(60) # 1 minute

except Exception as e:
    print(e)

'NoneType' object has no attribute 'find'


In [48]:
df = pd.DataFrame(results)

In [49]:
df.to_pickle("../MainFiles/inFiles/inFiles/inFiles/2705-2707Daryo.pickle")